In [ ]:
import os
import sys

SPARK_HOME = "/usr/lib/spark3"
PYSPARK_PYTHON = "/opt/conda/envs/dsenv/bin/python"
os.environ["PYSPARK_PYTHON"]= PYSPARK_PYTHON
os.environ["PYSPARK_DRIVER_PYTHON"]= PYSPARK_PYTHON
os.environ["SPARK_HOME"] = SPARK_HOME

PYSPARK_HOME = os.path.join(SPARK_HOME, "python/lib")
sys.path.insert(0, os.path.join(PYSPARK_HOME, "py4j-0.10.9.5-src.zip"))
sys.path.insert(0, os.path.join(PYSPARK_HOME, "pyspark.zip"))

# Apache Spark lost chapters. RDD, CLI, UI
# /opt/shared/
### 1. Смотрим в Spark UI
### 2. Исследуем Spark CLI
### 3. Пробегаемся по Spark RDD

# Dataframes, UI, Tuning

## Оптимизации Spark

### А какие у нас есть еще опции?
[Очень много](https://spark.apache.org/docs/3.3.0/configuration.html)

In [ ]:
import pyspark
from pyspark import SparkConf
from pyspark.sql import SparkSession

In [ ]:
conf = SparkConf()
conf.set('spark.ui.port', '14099')
conf.set('spark.driver.memory', '2G')
conf.set('spark.executor.memory', '2G')

In [ ]:
for p in sorted(conf.getAll(), key=lambda p: p[0]):
    print(p)

In [ ]:
for p in sorted(spark.sparkContext.getConf().getAll(), key=lambda p: p[0]):
    print(p)

In [ ]:
spark = SparkSession.builder.config(conf=conf).appName("spark_next_part").getOrCreate()

### Кривые данные

In [ ]:
data = """{"a": 1, "b":2, "c":3}|{"a": 1, "b":2, "c":3}|{"a": 1, "b, "c":10}""".split('|')

In [ ]:
data

In [ ]:
corruptDF = (spark.read
  .option("mode", "PERMISSIVE")
  .option("columnNameOfCorruptRecord", "_corrupt_record")
  .json(spark.sparkContext.parallelize(data))
)
corruptDF.show()

На клетке выше Спарк обработал испорченные данные и поместил их в специальную колонку, остальное успешно обработал.
Это поведение по умолчаниб, мы ничего не применяли опции `mode` и `columnNameOfCorruptRecord`.

Есть три варианта обработки испорченных записей через опцию [`ParseMode`](https://github.com/apache/spark/blob/master/sql/catalyst/src/main/scala/org/apache/spark/sql/catalyst/util/ParseMode.scala#L34):

| `ParseMode` | Поведение |
|-------------|----------|
| `PERMISSIVE` | Складывает все испорченные записии в специальную колонку "_corrupt_record" (по умолчанию) |
| `DROPMALFORMED` | Игнорирует все испорченные записи |
| `FAILFAST` | Выкидывает исключение на первой же неправильной записи |

Следующая клетка уже выкидывает испорченные данные:

In [ ]:
corruptDF = (spark.read
  .option("mode", "DROPMALFORMED")
  .json(spark.sparkContext.parallelize(data))
)
corruptDF.show()

### Кэширование
Какие вообще у нас опции?


* `DataFrame.persist(..)`

In [ ]:
pyspark.sql.DataFrame.persist?

In [ ]:
pyspark.storagelevel.StorageLevel?

In [ ]:
log.count()

In [ ]:
log.cache()

In [ ]:
%%time
log.count()

In [ ]:
log.unpersist()

In [ ]:
spark.catalog.clearCache()

### Spark UI
* Список запланированнных, исполняемых и исполненных этапов (stages) и задач (tasks)
* Информация о размере RDD и использовании памяти
* Информация об окружении
* Информация об экзекуторах
* Все задачи Spark SQL

### Давайте заглянем!

In [ ]:
spark

## Apache Spark RDD

### Что такое RDD?
Resilient Distributed Dataset, основа основ Apache Spark, низкоуровненая базовая абстракция, в которую превратится ваш код на DataFrame API

Из чего состоит RDD
* Dependencies
* Partitions
* Functions
* (если определен) Partitioner
* (если определен) Locations

Transformations и Actions точно так же работают и в RDD!

### Зачем использовать RDD?
* Нужен функционал, которого не оказалось в датафреймах 
* У вас есть старая кодовая база с RDD
* Вам нужен контроль за данными и за распределенными переменными
* Вы хотите добиться высокой производительности (**не в Python!**)

In [ ]:
spark.range(10).rdd

In [ ]:
spark.range(10).toDF("id").rdd.map(lambda row: row[0])

Всегда можно конвертировать туда и обратно!

In [ ]:
spark.range(10).rdd.toDF()

In [ ]:
myCollection = "Big Data and Data Engineering for AI Masters"\
.split(" ")
words = spark.sparkContext.parallelize(myCollection, 2)

In [ ]:
words

In [ ]:
words.setName("bigdata")

In [ ]:
words.name()

#### Transformations

In [ ]:
# distinct
words.distinct().count()

In [ ]:
# filter
def startsWithD(word):
    return word.startswith("D")

In [ ]:
words.filter(lambda word: startsWithD(word)).collect()

In [ ]:
# map
words2 = words.map(lambda word: (word, word[0], word.startswith("D")))

In [ ]:
words2.filter(lambda record: record[2]).take(5)

In [ ]:
# flatMap
words.flatMap(lambda word: list(word)).take(7)

In [ ]:
# sort
words.sortBy(lambda word: len(word) * -1).take(2)

In [ ]:
# Random Splits

In [ ]:
fiftyFiftySplit = words.randomSplit([0.5, 0.5])

In [ ]:
fiftyFiftySplit[0].collect()

In [ ]:
fiftyFiftySplit[1].collect()

#### Actions

In [ ]:
# reduce
spark.sparkContext.parallelize(range(1, 21)).reduce(lambda x, y: x + y)

In [ ]:
def wordLengthReducer(leftWord, rightWord):
    if len(leftWord) > len(rightWord):
        return leftWord
    else:
        return rightWord

In [ ]:
words.reduce(wordLengthReducer)

In [ ]:
# count
words.count()

In [ ]:
# countApprox
confidence = 0.95
timeoutMilliseconds = 100
words.countApprox(timeoutMilliseconds, confidence)

In [ ]:
words.countApprox?

In [ ]:
# countApproxDistinct
words.countApproxDistinct(0.05)

In [ ]:
words.countApproxDistinct?

In [ ]:
# countByValue
words.countByValue()

In [ ]:
# first
words.first()

In [ ]:
# max and min
spark.sparkContext.parallelize(range(1,20)).max()

In [ ]:
spark.sparkContext.parallelize(range(1,20)).min()

In [ ]:
# take
words.take(5)

In [ ]:
words.takeOrdered(5)

In [ ]:
words.top(5)

In [ ]:
words.takeSample(True, 10, 42)

In [ ]:
words.takeSample?

#### Caching

In [ ]:
words.cache()

In [ ]:
words.getStorageLevel()

In [ ]:
# сheckpointing
spark.sparkContext.setCheckpointDir("/tmp/rdd_checkpoint")

In [ ]:
words.checkpoint()

In [ ]:
#piping
words.pipe("wc -l").collect()

In [ ]:
words.getNumPartitions()

In [ ]:
# mapPartitions
words.mapPartitions(lambda part: [1]).sum()

In [ ]:
def indexedFunc(partitionIndex, withinPartIterator):
    return ["partition: {} => {}".format(partitionIndex, x) for x in withinPartIterator]
words.mapPartitionsWithIndex(indexedFunc).collect()

#### Key-Value RDDs
Многие методы подразумевают K-V структуру для выполнения.  
Эта группа выглядит как **"что-то"ByKey**

In [ ]:
# keyBy
keyword = words.keyBy(lambda word: word.lower()[0])

In [ ]:
keyword.take(5)

In [ ]:
# Mapping over Values
keyword.mapValues(lambda word: word.upper()).collect()

In [ ]:
#flatMap
keyword.flatMapValues(lambda word: word.upper()).collect()

In [ ]:
# Извлекаем K-V
keyword.keys().collect()

In [ ]:
keyword.values().collect()

In [ ]:
# lookup
keyword.lookup("d")

In [ ]:
# sampleByKey
import random
distinctChars = words.flatMap(lambda word: list(word.lower())).distinct()\
.collect()
sampleMap = dict(map(lambda c: (c, random.random()), distinctChars))
words.map(lambda word: (word.lower()[0], word))\
.sampleByKey(True, sampleMap, 6).collect()

#### Aggregations

In [ ]:
chars = words.flatMap(lambda word: word.lower())
KVcharacters = chars.map(lambda letter: (letter, 1))
def maxFunc(left, right):
    return max(left, right)
def addFunc(left, right):
    return left + right
nums = spark.sparkContext.parallelize(range(1,31), 5)

In [ ]:
# countByKey
KVcharacters.countByKey()

In [ ]:
# groupByKey
from functools import reduce
KVcharacters.groupByKey().map(lambda row: (row[0], reduce(addFunc, row[1])))\
.collect()

In [ ]:
# reduceByKey
KVcharacters.reduceByKey(addFunc).collect()

In [ ]:
# aggregate
nums.aggregate(0, maxFunc, addFunc)

In [ ]:
#treeAggregate
depth = 3
nums.treeAggregate(0, maxFunc, addFunc, depth)

In [ ]:
nums.treeAggregate?

In [ ]:
# aggregateByKey
KVcharacters.aggregateByKey(0, addFunc, maxFunc).collect()

In [ ]:
# combineByKey
def valToCombiner(value):
    return [value]
def mergeValuesFunc(vals, valToAppend):
    vals.append(valToAppend)
    return vals
def mergeCombinerFunc(vals1, vals2):
    return vals1 + vals2
outputPartitions = 6
KVcharacters.combineByKey(valToCombiner, mergeValuesFunc, mergeCombinerFunc, outputPartitions)\
.collect()

In [ ]:
# foldByKey
KVcharacters.foldByKey(0, addFunc).collect()

#### Joins

In [ ]:
# joins
distinctChars = words.flatMap(lambda word: word.lower()).distinct()
keyedChars = distinctChars.map(lambda c: (c, random.random()))
outputPartitions = 10
KVcharacters.join(keyedChars).count()
KVcharacters.join(keyedChars, outputPartitions).count()

In [ ]:
# zips
numRange = spark.sparkContext.parallelize(range(8), 2)
words.zip(numRange).collect()

#### Controlling Partitions

In [ ]:
# coalesce
words.coalesce(1).getNumPartitions()

In [ ]:
# repartition
words.repartition(10).getNumPartitions()

#### Broadcast Variables

In [ ]:
supplementalData = {"Big": 10000, "Data": -100, "Engineering": 400, "AI": 100}

In [ ]:
suppBroadcast = spark.sparkContext.broadcast(supplementalData)

In [ ]:
suppBroadcast.value

In [ ]:
words.map(lambda word: (word, suppBroadcast.value.get(word, 0)))\
.sortBy(lambda wordPair: wordPair[1])\
.collect()

#### Accumulators

In [ ]:
accBest = spark.sparkContext.accumulator(0)

In [ ]:
schema = StructType(fields=[
    StructField("userId", IntegerType()),
    StructField("movieId", IntegerType()),
    StructField("rating", FloatType()),
    StructField("timestamp", TimestampType()),
])

In [ ]:
df = spark.read\
          .schema(schema)\
          .format("csv")\
          .option("sep", ",")\
          .option("header", "true")\
          .load("/datasets/movielens/ratings.csv")

In [ ]:
def accBestFunc(movie_row):
    rating = movie_row["rating"]
    if rating >= 4:
        accBest.add(1)

In [ ]:
df.rdd.getNumPartitions()

In [ ]:
df.foreach(lambda movie_row: accBestFunc(movie_row))

In [ ]:
accBest.value

In [ ]:
df.count()

## Spark CLI

In [ ]:
!spark3-submit --help

In [ ]:
!cat ~/spark_example.py

In [ ]:
!cat ~/spark_example.sh

In [ ]:
! ~/spark_example.sh

In [ ]:
spark.stop()